In [30]:
import pandas as pd
df = pd.read_csv(r"C:\Users\maxd2\OneDrive - Universitaet St.Gallen\Desktop\DSF\Schallmessungen_smartcity.csv", sep=";")

In [31]:
df_sorted = df[df["Station"] == "Feldbergstrasse"]

df_sorted.head(5)

print(len(df_sorted))


235188


In [32]:
df_sorted_date = df_sorted.sort_values(by='Zeitstempel')

first_observation = df_sorted_date.head(1)
last_observation = df_sorted_date.tail(1)

print("First Observation:")
print(first_observation)

print("\nLast Observation:")
print(last_observation)

First Observation:
                Station                Zeitstempel  Wert             Geopunkt  \
125227  Feldbergstrasse  2020-06-24T12:14:00+00:00  69.4  47.567005, 7.594749   

        Geographische Breite  Geographische Länge               EUI  
125227             47.567005             7.594749  0004A30B00F156E2  

Last Observation:
                 Station                Zeitstempel  Wert  \
1804366  Feldbergstrasse  2024-11-21T15:49:00+00:00  24.1   

                    Geopunkt  Geographische Breite  Geographische Länge  \
1804366  47.567005, 7.594749             47.567005             7.594749   

                      EUI  
1804366  0004A30B00F156E2  


In [33]:
df_sorted_date['Zeitstempel'] = pd.to_datetime(df_sorted_date['Zeitstempel'])
df_filtered = df_sorted_date[df_sorted_date['Zeitstempel'] >= '2022-01-01']

df_filtered.head(100)

,Station,Zeitstempel,Wert,Geopunkt,Geographische Breite,Geographische Länge,EUI
696010,Feldbergstrasse,2022-01-01 00:04:00+00:00,72.0,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
1298200,Feldbergstrasse,2022-01-01 00:10:00+00:00,69.0,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
1298201,Feldbergstrasse,2022-01-01 00:15:00+00:00,67.6,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
1298202,Feldbergstrasse,2022-01-01 00:21:00+00:00,68.4,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
696011,Feldbergstrasse,2022-01-01 00:26:00+00:00,74.6,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
...,...,...,...,...,...,...,...
696042,Feldbergstrasse,2022-01-01 09:05:00+00:00,66.5,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
1298225,Feldbergstrasse,2022-01-01 09:11:00+00:00,65.9,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
203290,Feldbergstrasse,2022-01-01 09:16:00+00:00,66.2,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2
1298226,Feldbergstrasse,2022-01-01 09:22:00+00:00,64.8,"47.567005, 7.594749",47.567005,7.594749,0004A30B00F156E2


In [34]:
Schallmessungen = df_filtered.drop(columns=["Geopunkt", "Geographische Breite", "Geographische Länge", "EUI", "Station"])

Schallmessungen.rename(columns={'Zeitstempel': 'Datum'}, inplace=True)
#Schallmessungen['Datum'] = Schallmessungen['Datum'].dt.tz_localize(None)

Schallmessungen.set_index('Datum', inplace=True)



In [35]:
Schallmessungen.head()

,Wert
Datum,
2022-01-01 00:04:00+00:00,72.0
2022-01-01 00:10:00+00:00,69.0
2022-01-01 00:15:00+00:00,67.6
2022-01-01 00:21:00+00:00,68.4
2022-01-01 00:26:00+00:00,74.6


In [36]:
Schallmessungen.index = pd.to_datetime(Schallmessungen.index)
Schallmessungen_hourly = Schallmessungen["Wert"].resample('h').mean().round(4)
Schallmessungen_hourly = Schallmessungen_hourly.to_frame()
Schallmessungen_hourly.reset_index(inplace=True)
Schallmessungen_hourly.head()

nan_counts = Schallmessungen_hourly.isnull().sum()
print(nan_counts)



Datum       0
Wert     3129
dtype: int64


In [41]:
#fill up the missing values with a smoothing function

def smooth_missing_values(df, window=3, min_periods=1):
    """
    Fill missing values in the 'Traffic' column using a rolling mean with a given window size.
    
    Parameters:
    - df: DataFrame with the 'Traffic' column containing missing values
    - window: the size of the moving window for the rolling mean (default is 3)
    - min_periods: the minimum number of observations required to calculate the mean (default is 1)
    
    Returns:
    - DataFrame with missing values filled using rolling mean
    """
    # Assuming the DataFrame has 'Datum' and 'Hour' as index or columns
    df['Wert'] = df['Wert'].fillna(df['Wert'].rolling(window=window, min_periods=min_periods).mean())
    
    return df


Schallmessungen_hourly = smooth_missing_values(Schallmessungen_hourly)

nan_counts = Schallmessungen_hourly.isnull().sum()
print(nan_counts)

Schallmessungen_hourly.isnull().head()

Datum       0
Wert     2529
dtype: int64


,Datum,Wert
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False


In [27]:
#standardise the columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Schallmessungen_hourly["Wert"] = scaler.fit_transform(Schallmessungen_hourly[["Wert"]])

Schallmessungen_hourly["Wert"].mean(), Schallmessungen_hourly["Wert"].std()
#Schallmessungen_hourly["Wert"].head()

(np.float64(-6.987819853811409e-17), np.float64(1.0000219526919514))

In [28]:
Schallmessungen_hourly.head()

,Datum,Wert
0,2022-01-01 00:00:00+00:00,1.165048
1,2022-01-01 01:00:00+00:00,1.099469
2,2022-01-01 02:00:00+00:00,1.061739
3,2022-01-01 03:00:00+00:00,1.036424
4,2022-01-01 04:00:00+00:00,1.056021


In [29]:
#hopefully useless
#checking for missing values

# Create a date range from the minimum to the maximum date in the 'Datum' column
date_range = pd.date_range(start=Schallmessungen_hourly['Datum'].min(), end=Schallmessungen_hourly['Datum'].max(), freq='D')

# Create a DataFrame with all the dates in the range
all_dates = pd.DataFrame({'Datum': date_range})

# Merge this new DataFrame with the original to find missing dates
missing_dates = pd.merge(all_dates, Schallmessungen_hourly, on='Datum', how='left', indicator=True)

# Rows that are missing in the original DataFrame will have '_merge' == 'left_only'
missing_rows = missing_dates[missing_dates['_merge'] == 'left_only']

# Output the missing rows (if any)
if not missing_rows.empty:
    print(f"Missing rows for the following dates: {missing_rows['Datum'].tolist()}")
else:
    print("No rows are missing.")

No rows are missing.


Datum       0
Wert     3129
dtype: int64


In [12]:
Schallmessungen_hourly.to_csv(r"C:\Users\maxd2\OneDrive - Universitaet St.Gallen\Dokumente\GitHub\Its-Wekk\4 - Data\Final_Data\Cleaned\Schallmessungen_cleaned.csv", sep=";")


In [2]:
import pandas as pd
Stromverbrauch = pd.read_csv(r"C:\Users\maxd2\OneDrive - Universitaet St.Gallen\Desktop\DSF\Stromverbrauch_15minIntervall.csv", sep=";")

In [3]:
Stromverbrauch.tail()


Stromverbrauch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451915 entries, 0 to 451914
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Start der Messung         451915 non-null  object 
 1   Start der Messung (Text)  451915 non-null  object 
 2   Stromverbrauch            451915 non-null  float64
 3   Grundversorgte Kunden     153971 non-null  float64
 4   Freie Kunden              148019 non-null  float64
 5   Jahr                      451915 non-null  int64  
 6   Monat                     451915 non-null  int64  
 7   Tag                       451915 non-null  int64  
 8   Wochentag                 451915 non-null  int64  
 9   Tag des Jahres            451915 non-null  int64  
 10  Quartal                   451915 non-null  int64  
 11  Woche des Jahres          451915 non-null  int64  
dtypes: float64(3), int64(7), object(2)
memory usage: 41.4+ MB


In [6]:
Stromverbrauch_cut = Stromverbrauch.drop(columns=["Start der Messung (Text)", "Grundversorgte Kunden", "Freie Kunden", "Jahr", "Monat", "Tag", "Wochentag","Tag des Jahres", "Quartal", "Woche des Jahres"])

Stromverbrauch_cut.rename(columns={'Start der Messung': 'Datum'}, inplace=True)

Stromverbrauch_cut['Datum'] = pd.to_datetime(Stromverbrauch_cut['Datum'], errors='coerce', utc=True)

#Stromverbrauch_cut['Datum'] = Stromverbrauch_cut['Datum'].dt.tz_localize(None)

Stromverbrauch_cut = Stromverbrauch_cut.sort_values(by='Datum')


Stromverbrauch_cut = Stromverbrauch_cut[Stromverbrauch_cut['Datum'] >= '2022-01-01']

Stromverbrauch_cut.set_index('Datum', inplace=True)



In [8]:
Stromverbrauch_hourly = Stromverbrauch_cut["Stromverbrauch"].resample('h').mean().round(4)
Stromverbrauch_hourly = Stromverbrauch_hourly.to_frame()
Stromverbrauch_hourly.reset_index(inplace=True)
Stromverbrauch_hourly.head()

,Datum,Stromverbrauch
0,2022-01-01 00:00:00+00:00,26954.3115
1,2022-01-01 01:00:00+00:00,25162.1338
2,2022-01-01 02:00:00+00:00,24057.3085
3,2022-01-01 03:00:00+00:00,23895.0918
4,2022-01-01 04:00:00+00:00,24704.4652


#standardise 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Stromverbrauch_hourly["Stromverbrauch"] = scaler.fit_transform(Stromverbrauch_hourly[["Stromverbrauch"]])

Stromverbrauch_hourly["Stromverbrauch"].mean(), Stromverbrauch_hourly["Stromverbrauch"].std()

In [10]:
print(Stromverbrauch_hourly)

                          Datum  Stromverbrauch
0     2022-01-01 00:00:00+00:00       -1.116530
1     2022-01-01 01:00:00+00:00       -1.368609
2     2022-01-01 02:00:00+00:00       -1.524008
3     2022-01-01 03:00:00+00:00       -1.546825
4     2022-01-01 04:00:00+00:00       -1.432982
...                         ...             ...
25314 2024-11-20 18:00:00+00:00        1.049699
25315 2024-11-20 19:00:00+00:00        0.558946
25316 2024-11-20 20:00:00+00:00        0.131778
25317 2024-11-20 21:00:00+00:00       -0.205962
25318 2024-11-20 22:00:00+00:00       -0.579489

[25319 rows x 2 columns]


In [38]:
#hopefully useless
#checking for missing values

# Create a date range from the minimum to the maximum date in the 'Datum' column
date_range = pd.date_range(start=Stromverbrauch_hourly['Datum'].min(), end=Stromverbrauch_hourly['Datum'].max(), freq='D')

# Create a DataFrame with all the dates in the range
all_dates = pd.DataFrame({'Datum': date_range})

# Merge this new DataFrame with the original to find missing dates
missing_dates = pd.merge(all_dates, Stromverbrauch_hourly, on='Datum', how='left', indicator=True)

# Rows that are missing in the original DataFrame will have '_merge' == 'left_only'
missing_rows = missing_dates[missing_dates['_merge'] == 'left_only']

# Output the missing rows (if any)
if not missing_rows.empty:
    print(f"Missing rows for the following dates: {missing_rows['Datum'].tolist()}")
else:
    print("No rows are missing.")

No rows are missing.


In [11]:
Stromverbrauch_hourly.to_csv(r"C:\Users\maxd2\OneDrive - Universitaet St.Gallen\Dokumente\GitHub\Its-Wekk\4 - Data\Final_Data\Cleaned\Stromverbrauch_Basel_hourly_cleaned.csv")